# Clustering UK research projects to thematic topics using text data

This tutorial looks at the use of natural language processing and unsupervised learning to cluster projects in the Gateway to Research database to thematic topics.  

We will learn how to preprocess text data, transform words to numbers, use dimensionality reduction to project the documents on a 2D space and cluster them.

## What we will do:
* Import the Python modules that will be used in the analysis. 
* Read the Gateway to Research data.
* Do some very basic data cleaning and exploratory data analysis.
* Preprocess the abstracts of the research projects.
* Train word embeddings using word2vec and run some example queries.
* Create document vectors by averaging word vectors.
* Use t-SNE to reduce the dimensionality of the document vectors to 2D.
* Cluster the document vectors using Gaussian Mixtures.

## How is this tutorial structured:
For every section, I will highlight its **Goal** and **what we will do** to achieve it. Then, I will **explain the methods** we use, **what alternatives or additional thing we could do** and lastly, we will **run the code** together. Note that some code cells can "run" for a while, so we will run them first and then explain what they do.

**Feel free to ask questions at any point of the talk!**

In [ ]:
!pip install --upgrade git+https://github.com/nestauk/im_tutorials.git

In [ ]:
%load_ext autoreload

## Imports

### These are the Python libraries we will use throughout the tutorial. Let's describe the main ones:

**Numpy**: Widely used package for scientific computing with Python. If something is an array/matrix, we do it with Numpy!  
**Pandas**: Like Excel, but for Python.  
**Matplotlib**: The go-to data visualisation library in Python.  
**Scikit-learn**: One of the main libraries for machine learning in Python.  
**NLTK**: A well established Python package to work with human language data.

### We also created a few modules for this tutorial:  
**text_preprocessing**: Contains the functions to preprocess the research abstracts.  
**dim_reduction**: Wrappers and helper functions for t-SNE and Gaussian Mixtures.  

In [ ]:
%autoreload
%matplotlib inline

import ast
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from im_tutorials.data import *
from im_tutorials.utilities import flatten_lists
from im_tutorials.features.text_preprocessing import *
from im_tutorials.features.document_vectors import document_vector
from im_tutorials.features.dim_reduction import WrapTSNE, GaussianMixtureEval

## Read the data

The data for this project is stored as a CSV file on Amazon Web Services (AWS) S3, a static cloud file storage service. We can use pandas to pull the data directly into a DataFrame.

In [ ]:
# Read the data 
gtr_projects_df = gtr.gateway_to_research_projects()

# Drop duplicates
gtr_projects_df.drop_duplicates('project_id', inplace=True)

# Shape of our data
print('The Gateway to Research data has {} rows and {} columns'.format(gtr_projects_df.shape[0], gtr_projects_df.shape[1]))

## Basic exploratory data analysis

### Goal
Explore the dataset, do some very basic cleaning and prepare it for further analysis.

### We will:
* Find the missing values and remove them
* See how our data look like
* Plot how many project were submitted every year.
* Find which Research Councils have funded the most projects.
* Examine the length of the abstract of the research projects (we are creating new features here!).

All of the above will be done with Pandas and Matplotlib!

### Other things we could do:
* Spot outliers
* Normalise some data fields
* Basic statistics for the features

In [ ]:
# Missing values
gtr_projects_df.isnull().sum()

In [ ]:
# Drop missing data
gtr_projects_df.dropna(inplace=True)

In [ ]:
gtr_projects_df.head()

In [ ]:
f, (ax1, ax2) = plt.subplots(nrows=2, figsize=(15,8))

# ax1
# Count the projects per year and sort them from older to newer ones. 
gtr_projects_df.start_year.value_counts().reindex(sorted(gtr_projects_df.start_year.value_counts().index)).plot(ax=ax1)
ax1.set_title('Number of research projects', fontsize=15)

# ax2 
# Group by the data on 'start_year' and 'funder_name' and count the number of projects belonging to each of these groups.
gtr_projects_df.groupby(['start_year', 'funder_name']).count()['project_id'].unstack('funder_name').plot(kind='bar', stacked=True, ax=ax2)
ax2.set_title('Funded projects by Research Council', fontsize=15)

plt.tight_layout()
plt.show()

In [ ]:
# Find the average length of the abstracts
gtr_projects_df['abstract_texts_length'] = gtr_projects_df.abstract_texts.apply(lambda doc: len(doc))

# Plot
f, (ax1, ax2) = plt.subplots(nrows=2, figsize=(15,8))

# ax1
# Group by the projects on the 'start_year' and average the length of their research abstracts.
gtr_projects_df.groupby('start_year').mean()['abstract_texts_length'].plot(ax=ax1)
ax1.set_title('Average length of research abstracts', fontsize=15)

# ax2 
# Group by the projects on the 'start_year' and 'funder_name' and average the length of the research abstracts for each group.
gtr_projects_df.groupby(['start_year', 'funder_name']).mean()['abstract_texts_length'].unstack('funder_name').plot(kind='bar', stacked=True, ax=ax2)
ax2.set_title('Average length of research abstracts by Research Council', fontsize=15)

plt.tight_layout()
plt.show()

## Text preprocessing

### Goal

We will clean the abstract of every research project. 

### We will:
* Lowercase and tokenise the documents.
* Keep only tokens that have more than two characters.
* Filter out stop words, punctuation, numeric and non-English characters.
* Create n-grams.

### Some definitions

**Natural language processing**: It's a subfield of computer science and information engineering (and very influenced by linguistics) concerned with how to program computers to process and analyse natural language data.
**Tokenisation:** It is a particular kind of document segmentation. Segmentation breaks up text into smaller chunks or segments, with more focused information content. Tokens are instances of a sequence of characters in some particular document that are grouped together as a useful semantic unit for processing.  
**n-grams**: Sequence of concatenated tokens.
**Vocabulary:** A collection of the unique tokens found in a collection of documents.  
**One-hot-vectors:** A representation of tokens as binary vectors. The length of the vector is equal to the size of the vocabulary.  

<img src="../reports/figures/onehot.png" width="600">

### Other things that you could try:
* Change the regular expression used to tokenise the documents.
* Stem or lemmatise tokens based on their Part-of-Speech tag.

In [ ]:
%%time
# Process the abstracts
gtr_projects_df['processed_documents'] = build_ngrams([flatten_lists(tokenize_document(document)) for document in list(gtr_projects_df.abstract_texts)])

In [ ]:
# Binary column showing if a document has any tokens left after preprocessing
gtr_projects_df['is_Doc'] = gtr_projects_df.processed_documents.apply(lambda doc: 1 if len(doc) != 0 else 0)

# Keep only rows with a processed abstract
gtr_projects_df = gtr_projects_df[gtr_projects_df.is_Doc == 1]

## Create word vectors

### Goal
Represent the words with vectors.

### We will:

Use [word2vec](https://radimrehurek.com/gensim/models/word2vec.html) to create a dense vector representation of tokens and project them to a low-dimensional vector space. This is done in a way that similar tokens will be closer on the vector space.

### Why not using something simpler?
Few disadvantages of one-hot-vectors:
* Very high dimensionality -> takes longer to train ML models.
* They do not learn based on their context, meaning that we cannot find relations between tokens.

<img src="../reports/figures/onehotencoding.png">

### What's word2vec?

Word2Vec is a shallow neural network which takes as input a large collection of sentences and produces a vector space of words, typically of several hundred dimensions where each word is assigned to corresponding vector in the space. Word vectors are positioned in the vector space in a way that those sharing similar context are located in close proximity to one another in the high dimensional vector space.  

<img src="../reports/figures/word2vec.png" width="800">

Word2Vec is a particularly computationally-efficient predictive model for learning word embeddings from raw text.

### Architecture

<img src="../reports/figures/w2v_architectures.png" width="800">

### Word2vec: Few important hyperparameters
* **`size`**: Dimensionality of the word vectors (length of the hidden layer).
* **`min_count`**: Ignores all words with total frequency lower than this.
* **`window`**: Maximum distance between the current and predicted word within a sentence.
* **`learning rate`**: The learning rate used in training.
* We can choose between the **`skip-gram`** and **`Continuous BOW`** implementation of the algorithm.
* We can choose **`Hierachical Softmax`** or **`Negative Sampling`** to train of the algorithm.


### Other methods you could use:
* One-hot-encodings
* Use a pre-trained word2vec model
* [TF-IDF](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
* [GloVe](https://nlp.stanford.edu/projects/glove/)
* [fastText](https://fasttext.cc/)
* [ELMo](https://allennlp.org/elmo)

In [ ]:
%%time
w2v = gensim.models.Word2Vec(list(gtr_projects_df.processed_documents), size=300, window=10, min_count=2, iter=20)

### Some examples of word2vec

We can query the trained word2vec with a token from its vocabulary and find the ones that are most similar to it. The float number of every "row" shows the cosine similarity between the query and the token on the left of the score.

In [ ]:
w2v.wv.most_similar(['virtual_reality'])

In [ ]:
w2v.wv.most_similar(['blockchain'])

In [ ]:
w2v.wv.most_similar(['artificial_intelligence'])

In [ ]:
w2v.wv.most_similar(['machine_learning'])

In [ ]:
w2v.wv.most_similar(['big_data'])

In [ ]:
w2v.wv.most_similar(['social_sciences'])

In [ ]:
w2v.wv.most_similar(['particle_physics'])

## Create document vectors

### Goal 
Create document vectors!

### We will:
Average the word vectors of a document to create document vectors. The words that are not in word2vec model's vocabulary are ignored.

### Other methods you could use:
* Averaged word vectors weighted by their TF-IDF.
* [doc2vec](https://radimrehurek.com/gensim/models/doc2vec.html)
* [sent2vec](https://rare-technologies.com/sent2vec-an-unsupervised-approach-towards-learning-sentence-embeddings/)

In [ ]:
# Take a sample of the data to make the second part of the tutorial run faster
gtr_projects_df = gtr_projects_df.sample(8000)

# Average word vectors
doc_vecs = [document_vector(doc=processed_document, word2vec_model=w2v) for processed_document in list(gtr_projects_df.processed_documents)]

## Cluster vectorised documents 

### Goal
Reduce the dimensionality of the document vectors and cluster them.

### We will:
* Find the cosine distance of the document vector matrix.
* Reduce its dimensionality to 2D using [t-SNE](https://lvdmaaten.github.io/tsne/).
* Cluster the points of the 2D space using [Gaussian Mixtures](https://scikit-learn.org/stable/modules/mixture.html)

### t-SNE
t-SNE is a nonlinear dimensionality reduction technique well-suited for embedding high-dimensional data in a low-dimensional space (usually 2D).  

In the high-dimensional space, t-SNE creates a probability distribution that dictates the relationships between various neighbouring points. Then, it tries to recreate a low dimensional space that follows that probability distribution as best as possible.

<img src="../reports/figures/tsne.png" width="800">

#### t-SNE: Few important hyperparameters
* **`n_components`**: Dimension of the embedded space.
* **`perplexity`:** The perplexity is related to the number of nearest neighbors that is used in other manifold learning algorithms. Larger datasets usually require a larger perplexity. Consider selecting a value between 5 and 50. The choice is not extremely critical since t-SNE is quite insensitive to this parameter. The performance of t-SNE is fairly robust under different settings of the perplexity. 
* **`learning_rate`:** If the learning rate is too high, the data may look like a ‘ball’ with any point approximately equidistant from its nearest neighbours. If the learning rate is too low, most points may look compressed in a dense cloud with few outliers. If the cost function gets stuck in a bad local minimum increasing the learning rate may help.

### Gaussian Mixtures
A Gaussian mixture model is a probabilistic model that assumes all the data points are generated from a mixture of a finite number of Gaussian distributions with unknown parameters.

<img alt="Credit: KazukiAmakawa [CC BY-SA 4.0 (https://creativecommons.org/licenses/by-sa/4.0)]" src="../reports/figures/gaussian_mixture.gif" width="800">

#### GMM: Few important hyperparameters
* **`n_components`**: The number of mixture components.
* **`covariance_type`**: The type of covariance to use.

<img src="../reports/figures/covariance_types.png" width="800">

### Other methods we could use:
* PCA (for dimensionality reduction)
* k-Means (for clustering)

In [ ]:
%%time
# Dimensionality reduction with t-SNE
ts = WrapTSNE()
tsne_space = ts.reduce_dimensions(doc_vecs, n_iter=1000)

In [ ]:
# Clustering with GMM
# Instantiate the class using the 2D space from t-SNE
np.random.seed(42)
gmm_eval = GaussianMixtureEval(tsne_space)

best_gmm, bic = gmm_eval.fit_eval(max_components=40)
print('BEST GMM MODEL: {}'.format(best_gmm))

In [ ]:
# Predict token clusters based on best_gmm
token_labels = best_gmm.predict(tsne_space)

In [ ]:
plt.figure(figsize=(15,15))
plt.scatter(tsne_space[:,0], tsne_space[:,1], c=token_labels)
plt.title('Document vector space', fontsize=15)
plt.show()

## Conclusions

In this tutorial, we've seen how to preprocess text data, find the vector representation of documents using word embeddings, reduced their dimensionality to 2D with t-SNE and cluster them with Gaussian Mixtures.

### Where can we go from here?
* Tryout the methods we mentioned at each part of the tutorial. Most of them will definitely improve the results!
* The methods we showcased can -in principle- be used with any text data. How would you use them in your work?

### Few things to consider
* **Scalability of the methods.** We decided to use less documents in t-SNE and GMMs in order to reduce the computing time.
* It's **difficult to evaluate** the results of unsupervised learning methods. 